In [1]:
!python -m pip install -U pip setuptools wheel



In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128


Looking in indexes: https://download.pytorch.org/whl/cu128
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/2.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 GB 110.2 MB/s eta 0:00:26
    --------------------------------------- 0.0/2.9 GB 114.0 MB/s eta 0:00:25
    --------------------------------------- 0.1/2.9 GB 115.2 MB/s eta 0:00:25
   - -------------------------------------- 0.1/2.9 GB 115.8 MB/s eta 0:00:24
   - -------------------------------------- 0.1/2.9 GB 116.1 MB/s eta 0:00:24
   - -------------------------------------- 0.1/2.9 GB 116.4 MB/s eta 0:00:24
   -- ------------------------------------- 0.2/2.9 GB 116.5 MB/s eta 0:00:24
   -- ------------------------------------- 0.2/2.9 GB 117.8 MB/s eta

In [3]:
!pip install -U "gymnasium[atari]"


  Using cached gymnasium-1.2.2-py3-none-any.whl.metadata (10 kB)
  Using cached cloudpickle-3.1.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
Using cached gymnasium-1.2.2-py3-none-any.whl (952 kB)
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ------------------------------------ --- 3.1/3.5 MB 20.5 MB/s eta 0:00:01
   ---------------------------------------- 3.5/3.5 MB 18.7 MB/s  0:00:00
Using cached cloudpickle-3.1.2-py3-none-any.whl (22 kB)
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)

   ------------------------------ --------- 3/4 [gymnasium]
   ------------------------------ --------- 3/4 [gymnasium]
   ---------------------------------------- 4/4 [gymnasium]



In [4]:
!pip install "stable-baselines3[extra]"

  Using cached stable_baselines3-2.7.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached rich-14.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached stable_baselines3-2.7.1-py3-none-any.whl (188 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   -------- ------------------------------- 2.4/10.6 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 31.6 MB/s  0:00:00
Using cached rich-14.2.0-py3-none-any.whl (243 kB)
Using cached markdown_it_py-4.0.0-py3-none-any.whl 

In [1]:
import torch

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))
    print("torch cuda version:", torch.version.cuda)

torch: 2.9.1+cu128
cuda available: True
gpu: NVIDIA GeForce RTX 4070
torch cuda version: 12.8


In [2]:
import gymnasium as gym
import ale_py  # registers ALE envs

env = gym.make("ALE/Pong-v5")  # no rendering needed for a smoke test
obs, info = env.reset()
print("obs shape:", obs.shape)

for _ in range(200):
    obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
    if terminated or truncated:
        obs, info = env.reset()

env.close()
print("ok")

obs shape: (210, 160, 3)
ok


# DQN Training Setup

**To start training, run these cells in order:**
1. Cell with Model class (torch imports + Model definition)
2. Cell with ReplayBuffer class
3. Cell with Environment Preprocessing (PreprocessAtari, FrameStack)
4. Cell with DQNAgent class
5. **Cell with Training Loop** - This is the one that actually starts training!

**Training will:**
- Run for 1,000,000 steps
- Start learning after 10,000 steps of exploration
- Save model every 100,000 steps to `models/dqn_pong.pth`
- Show progress with epsilon, loss, and episode rewards


In [2]:
# Ensure torch is imported (if not already imported in earlier cells)
import torch
import torch.nn as nn
import torch.nn.functional as F


In [17]:
# FIXED: PreprocessAtari wrapper with proper reset and step methods
import gymnasium as gym
import numpy as np
from gymnasium import Wrapper
import cv2
from collections import deque

class PreprocessAtari(Wrapper):
    """
    Preprocesses Atari frames: resize to 84x84, convert to grayscale.
    """
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(84, 84), dtype=np.uint8
        )
    
    def observation(self, obs):
        # Convert to grayscale if needed
        if len(obs.shape) == 3:
            obs = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
        
        # Resize to 84x84
        obs = cv2.resize(obs, (84, 84), interpolation=cv2.INTER_AREA)
        
        return obs
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        return self.observation(obs), info
    
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        return self.observation(obs), reward, terminated, truncated, info

class FrameStack(Wrapper):
    """
    Stacks the last n frames together.
    """
    def __init__(self, env, n_frames=4):
        super().__init__(env)
        self.n_frames = n_frames
        self.frames = deque(maxlen=n_frames)
        
        # Update observation space
        obs_shape = env.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0, high=255, 
            shape=(n_frames, obs_shape[0], obs_shape[1]), 
            dtype=np.uint8
        )
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        # Fill the frame stack with the first frame
        for _ in range(self.n_frames):
            self.frames.append(obs)
        return self._get_obs(), info
    
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_obs(), reward, terminated, truncated, info
    
    def _get_obs(self):
        # Stack frames: (n_frames, height, width)
        return np.stack(self.frames, axis=0)

print("✅ Preprocessing wrappers updated with proper reset/step methods!")


✅ Preprocessing wrappers updated with proper reset/step methods!


In [18]:
import random
from collections import deque, namedtuple

# Named tuple for storing experiences
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """
    Experience Replay Buffer for DQN.
    Stores and samples batches of experiences for training.
    """
    def __init__(self, capacity):
        """
        Args:
            capacity: Maximum number of experiences to store
        """
        self.buffer = deque(maxlen=capacity)
        self.capacity = capacity
    
    def push(self, state, action, reward, next_state, done):
        """Add an experience to the buffer."""
        experience = Experience(state, action, reward, next_state, done)
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        """Sample a batch of experiences from the buffer."""
        batch = random.sample(self.buffer, batch_size)
        
        # Unpack the batch
        states = torch.stack([e.state for e in batch])
        actions = torch.tensor([e.action for e in batch], dtype=torch.long)
        rewards = torch.tensor([e.reward for e in batch], dtype=torch.float32)
        next_states = torch.stack([e.next_state for e in batch])
        dones = torch.tensor([e.done for e in batch], dtype=torch.float32)
        
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        """Return the current size of the buffer."""
        return len(self.buffer)


In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import Wrapper
import cv2

class PreprocessAtari(Wrapper):
    """
    Preprocesses Atari frames: resize to 84x84, convert to grayscale, normalize.
    """
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(84, 84), dtype=np.uint8
        )
    
    def observation(self, obs):
        # Convert to grayscale if needed
        if len(obs.shape) == 3:
            obs = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
        
        # Resize to 84x84
        obs = cv2.resize(obs, (84, 84), interpolation=cv2.INTER_AREA)
        
        return obs
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        return self.observation(obs), info
    
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        return self.observation(obs), reward, terminated, truncated, info

class FrameStack(Wrapper):
    """
    Stacks the last n frames together.
    """
    def __init__(self, env, n_frames=4):
        super().__init__(env)
        self.n_frames = n_frames
        self.frames = deque(maxlen=n_frames)
        
        # Update observation space
        obs_shape = env.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0, high=255, 
            shape=(n_frames, obs_shape[0], obs_shape[1]), 
            dtype=np.uint8
        )
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        # Fill the frame stack with the first frame
        for _ in range(self.n_frames):
            self.frames.append(obs)
        return self._get_obs(), info
    
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_obs(), reward, terminated, truncated, info
    
    def _get_obs(self):
        # Stack frames: (n_frames, height, width)
        return np.stack(self.frames, axis=0)


## 🚨 Problem Diagnosis

**Current Status:** Agent stuck at -21.00 reward (random play) after 1M steps - **NOT LEARNING**

**Issues Identified:**
1. ❌ Epsilon decays too fast (reaches 0.01 by step 10k, then stays there)
2. ❌ No optimistic initialization - agent doesn't explore enough initially
3. ❌ Q-values might be converging to bad local minimum
4. ❌ Loss is very small but rewards don't improve (overfitting to bad policy)
5. ❌ No Double DQN (helps with overestimation bias)

**Solutions:**
- ✅ Optimistic initialization (initialize Q-values high to encourage exploration)
- ✅ Slower epsilon decay (explore longer)
- ✅ Double DQN (reduce overestimation)
- ✅ Better learning rate schedule
- ✅ Reward clipping/normalization


In [22]:
# IMPROVED DQN Agent with Optimistic Initialization and Double DQN
import torch.optim as optim
import math
import random

class ImprovedDQNAgent:
    """
    Improved DQN Agent with:
    - Optimistic initialization (early exploration, late exploitation)
    - Double DQN (reduces overestimation bias)
    - Better epsilon decay schedule
    - Learning rate scheduling
    """
    def __init__(
        self,
        state_shape,
        n_actions,
        device='cuda',
        lr=1e-4,
        gamma=0.99,
        epsilon_start=1.0,
        epsilon_end=0.01,
        epsilon_decay=500000,  # MUCH slower decay - explore for longer
        target_update_freq=1000,
        buffer_size=100000,
        batch_size=32,
        optimistic_init=10.0  # Initialize Q-values optimistically
    ):
        """
        Args:
            optimistic_init: Initial Q-value (high = more exploration early on)
        """
        self.device = device
        self.n_actions = n_actions
        self.gamma = gamma
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.target_update_freq = target_update_freq
        self.batch_size = batch_size
        self.steps = 0
        self.optimistic_init = optimistic_init
        
        # Create Q-network and target network
        n_frames = state_shape[0]
        self.q_network = Model(n_actions, n_frames).to(device)
        self.target_network = Model(n_actions, n_frames).to(device)
        
        # OPTIMISTIC INITIALIZATION: Initialize final layer to output high Q-values
        # This encourages exploration early on
        with torch.no_grad():
            self.q_network.fc2.weight.data.fill_(0.0)
            self.q_network.fc2.bias.data.fill_(optimistic_init)
            self.target_network.fc2.weight.data.fill_(0.0)
            self.target_network.fc2.bias.data.fill_(optimistic_init)
        
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        
        # Optimizer with learning rate scheduling
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=200000, gamma=0.5)
        
        # Replay buffer
        self.replay_buffer = ReplayBuffer(buffer_size)
    
    def get_epsilon(self):
        """Calculate current epsilon - slower decay."""
        if self.steps < self.epsilon_decay:
            return self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
                   (1 - self.steps / self.epsilon_decay)
        else:
            return self.epsilon_end
    
    def select_action(self, state, training=True):
        """Select an action using epsilon-greedy policy."""
        if training and random.random() < self.get_epsilon():
            return random.randrange(self.n_actions)
        
        # Convert state to tensor and add batch dimension
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device) / 255.0
        
        with torch.no_grad():
            q_values = self.q_network(state_tensor)
            action = q_values.argmax(1).item()
        
        return action
    
    def store_transition(self, state, action, reward, next_state, done):
        """Store a transition in the replay buffer."""
        # Clip rewards to [-1, 1] for stability
        reward = np.clip(reward, -1.0, 1.0)
        
        # Convert to tensors
        state_tensor = torch.FloatTensor(state).to(self.device) / 255.0
        next_state_tensor = torch.FloatTensor(next_state).to(self.device) / 255.0
        
        self.replay_buffer.push(state_tensor, action, reward, next_state_tensor, done)
        self.steps += 1
    
    def train_step(self):
        """Perform one training step with Double DQN."""
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        # Sample batch from replay buffer
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        # Normalize states
        states = states / 255.0
        next_states = next_states / 255.0
        
        # Compute Q(s, a)
        q_values = self.q_network(states)
        q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # DOUBLE DQN: Use main network to select action, target network to evaluate
        with torch.no_grad():
            # Select best action using main network
            next_q_values_main = self.q_network(next_states)
            next_actions = next_q_values_main.argmax(1)
            
            # Evaluate using target network
            next_q_values_target = self.target_network(next_states)
            next_q_value = next_q_values_target.gather(1, next_actions.unsqueeze(1)).squeeze(1)
            
            target_q_value = rewards + (1 - dones) * self.gamma * next_q_value
        
        # Compute loss
        loss = F.mse_loss(q_value, target_q_value)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 10)
        self.optimizer.step()
        
        # Update learning rate
        self.scheduler.step()
        
        # Update target network periodically
        if self.steps % self.target_update_freq == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())
        
        return loss.item()
    
    def save(self, filepath):
        """Save the model."""
        torch.save({
            'q_network': self.q_network.state_dict(),
            'target_network': self.target_network.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'scheduler': self.scheduler.state_dict(),
            'steps': self.steps,
        }, filepath)
    
    def load(self, filepath):
        """Load the model."""
        checkpoint = torch.load(filepath, map_location=self.device)
        self.q_network.load_state_dict(checkpoint['q_network'])
        self.target_network.load_state_dict(checkpoint['target_network'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        if 'scheduler' in checkpoint:
            self.scheduler.load_state_dict(checkpoint['scheduler'])
        self.steps = checkpoint['steps']

print("✅ Improved DQN Agent with optimistic initialization and Double DQN ready!")


✅ Improved DQN Agent with optimistic initialization and Double DQN ready!


In [23]:
# Training with Improved Agent
import ale_py
import os
from tqdm import tqdm
import numpy as np

# Create environment with preprocessing
env = gym.make("ALE/Pong-v5")
env = PreprocessAtari(env)
env = FrameStack(env, n_frames=4)

# Get environment info
state_shape = env.observation_space.shape
n_actions = env.action_space.n

print(f"State shape: {state_shape}")
print(f"Number of actions: {n_actions}")

# Create IMPROVED agent
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

agent = ImprovedDQNAgent(
    state_shape=state_shape,
    n_actions=n_actions,
    device=device,
    lr=1e-4,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.01,
    epsilon_decay=500000,  # Explore for 500k steps (was 10k!)
    target_update_freq=1000,
    buffer_size=100000,
    batch_size=32,
    optimistic_init=10.0  # High initial Q-values = more exploration
)

# Training parameters
total_steps = 2_000_000  # Train longer
learning_starts = 10_000
train_freq = 4
save_freq = 200_000
eval_freq = 50_000

# Statistics
episode_rewards = []
episode_lengths = []
current_episode_reward = 0
current_episode_length = 0

# Load existing model if available
model_path = "models/dqn_pong_improved.pth"
if os.path.exists(model_path):
    agent.load(model_path)
    print(f"✅ Loaded existing model from {model_path}")
    print(f"   Continuing from step {agent.steps}")
else:
    os.makedirs("models", exist_ok=True)
    print("🚀 Starting fresh training with improved agent")
    print("   - Optimistic initialization (explores more early)")
    print("   - Double DQN (reduces overestimation)")
    print("   - Slower epsilon decay (explores for 500k steps)")

# Training loop
state, info = env.reset()
pbar = tqdm(range(total_steps), desc="Training", initial=agent.steps if agent.steps < total_steps else 0)

for step in range(agent.steps, total_steps):
    # Select action
    action = agent.select_action(state, training=True)
    
    # Take step
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    
    # Store transition
    agent.store_transition(state, action, reward, next_state, done)
    
    # Update statistics
    current_episode_reward += reward
    current_episode_length += 1
    
    # Train
    if step >= learning_starts and step % train_freq == 0:
        loss = agent.train_step()
        if loss is not None:
            current_lr = agent.optimizer.param_groups[0]['lr']
            pbar.set_postfix({
                'epsilon': f'{agent.get_epsilon():.3f}',
                'loss': f'{loss:.4f}',
                'lr': f'{current_lr:.2e}',
                'avg_reward': f'{np.mean(episode_rewards[-10:]):.1f}' if len(episode_rewards) >= 10 else 'N/A'
            })
    
    # Handle episode end
    if done:
        episode_rewards.append(current_episode_reward)
        episode_lengths.append(current_episode_length)
        current_episode_reward = 0
        current_episode_length = 0
        state, info = env.reset()
    else:
        state = next_state
    
    # Save model periodically
    if step > 0 and step % save_freq == 0:
        agent.save(model_path)
        print(f"\n💾 Model saved at step {step:,}")
    
    # Evaluate and show progress
    if step > 0 and step % eval_freq == 0 and len(episode_rewards) >= 10:
        avg_reward = np.mean(episode_rewards[-10:])
        avg_length = np.mean(episode_lengths[-10:])
        print(f"\n📊 Step {step:,}: Avg reward (last 10): {avg_reward:.2f}, Avg length: {avg_length:.1f}, Epsilon: {agent.get_epsilon():.3f}")
        
        # Show learning progress
        if avg_reward > 0:
            print("   🎉🎉🎉 BREAKTHROUGH! Agent is winning!")
        elif avg_reward > -10:
            print("   🎯 Great progress! Agent is learning!")
        elif avg_reward > -15:
            print("   📈 Starting to improve!")
        elif avg_reward > -19:
            print("   📊 Better than random, keep going!")
        else:
            print("   ⏳ Still exploring...")
    
    pbar.update(1)

# Final save
agent.save(model_path)
env.close()
pbar.close()

print(f"\n✅ Training complete! Model saved to {model_path}")
print(f"Total episodes: {len(episode_rewards)}")
if len(episode_rewards) > 0:
    print(f"Final average reward (last 100): {np.mean(episode_rewards[-100:]):.2f}")
    print(f"Best average reward (last 10): {np.mean(episode_rewards[-10:]):.2f}")
    
    # Final assessment
    final_avg = np.mean(episode_rewards[-10:])
    if final_avg > 10:
        print("🌟🌟 EXCELLENT! Agent mastered the game!")
    elif final_avg > 0:
        print("🎯 SUCCESS! Agent is winning more than losing!")
    elif final_avg > -10:
        print("📈 Good progress! Agent is learning!")
    elif final_avg > -15:
        print("📊 Some improvement, but needs more training")
    else:
        print("⏳ Still needs work - may need hyperparameter tuning")


State shape: (4, 84, 84)
Number of actions: 6
Using device: cuda
🚀 Starting fresh training with improved agent
   - Optimistic initialization (explores more early)
   - Double DQN (reduces overestimation)
   - Slower epsilon decay (explores for 500k steps)


Training:   0%|          | 1399/2000000 [00:05<5:11:59, 106.76it/s]

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 4, 210, 160, 3]

Training:   0%|          | 1436/2000000 [00:21<5:11:59, 106.76it/s]

In [21]:
import torch.optim as optim
import math

class DQNAgent:
    """
    DQN Agent with experience replay and target network.
    """
    def __init__(
        self,
        state_shape,
        n_actions,
        device='cuda',
        lr=1e-4,
        gamma=0.99,
        epsilon_start=1.0,
        epsilon_end=0.01,
        epsilon_decay=10000,
        target_update_freq=1000,
        buffer_size=100000,
        batch_size=32
    ):
        """
        Args:
            state_shape: Shape of state (n_frames, height, width)
            n_actions: Number of possible actions
            device: Device to run on ('cuda' or 'cpu')
            lr: Learning rate
            gamma: Discount factor
            epsilon_start: Starting epsilon for epsilon-greedy
            epsilon_end: Final epsilon
            epsilon_decay: Steps to decay epsilon
            target_update_freq: Frequency to update target network
            buffer_size: Size of replay buffer
            batch_size: Batch size for training
        """
        self.device = device
        self.n_actions = n_actions
        self.gamma = gamma
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.target_update_freq = target_update_freq
        self.batch_size = batch_size
        self.steps = 0
        
        # Create Q-network and target network
        n_frames = state_shape[0]
        self.q_network = Model(n_actions, n_frames).to(device)
        self.target_network = Model(n_actions, n_frames).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()  # Target network is always in eval mode
        
        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        
        # Replay buffer
        self.replay_buffer = ReplayBuffer(buffer_size)
    
    def get_epsilon(self):
        """Calculate current epsilon for epsilon-greedy policy."""
        return self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
               math.exp(-1. * self.steps / self.epsilon_decay)
    
    def select_action(self, state, training=True):
        """
        Select an action using epsilon-greedy policy.
        
        Args:
            state: Current state (n_frames, height, width)
            training: If True, use epsilon-greedy; if False, use greedy
        
        Returns:
            Selected action
        """
        if training and random.random() < self.get_epsilon():
            return random.randrange(self.n_actions)
        
        # Convert state to tensor and add batch dimension
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device) / 255.0
        
        with torch.no_grad():
            q_values = self.q_network(state_tensor)
            action = q_values.argmax(1).item()
        
        return action
    
    def store_transition(self, state, action, reward, next_state, done):
        """Store a transition in the replay buffer."""
        # Convert to tensors
        state_tensor = torch.FloatTensor(state).to(self.device) / 255.0
        next_state_tensor = torch.FloatTensor(next_state).to(self.device) / 255.0
        
        self.replay_buffer.push(state_tensor, action, reward, next_state_tensor, done)
        self.steps += 1
    
    def train_step(self):
        """Perform one training step."""
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        # Sample batch from replay buffer
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        
        # Move to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        # Normalize states
        states = states / 255.0
        next_states = next_states / 255.0
        
        # Compute Q(s, a)
        q_values = self.q_network(states)
        q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Compute target Q-values
        with torch.no_grad():
            next_q_values = self.target_network(next_states)
            next_q_value = next_q_values.max(1)[0]
            target_q_value = rewards + (1 - dones) * self.gamma * next_q_value
        
        # Compute loss
        loss = F.mse_loss(q_value, target_q_value)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 10)
        self.optimizer.step()
        
        # Update target network periodically
        if self.steps % self.target_update_freq == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())
        
        return loss.item()
    
    def save(self, filepath):
        """Save the model."""
        torch.save({
            'q_network': self.q_network.state_dict(),
            'target_network': self.target_network.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'steps': self.steps,
        }, filepath)
    
    def load(self, filepath):
        """Load the model."""
        checkpoint = torch.load(filepath, map_location=self.device)
        self.q_network.load_state_dict(checkpoint['q_network'])
        self.target_network.load_state_dict(checkpoint['target_network'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.steps = checkpoint['steps']


In [13]:
# Quick verification - Run this before training to make sure everything is set up
try:
    # Check if all classes are defined
    assert 'Model' in globals(), "Model class not found - run the Model cell first!"
    assert 'ReplayBuffer' in globals(), "ReplayBuffer class not found - run the ReplayBuffer cell first!"
    assert 'PreprocessAtari' in globals(), "PreprocessAtari class not found - run the preprocessing cell first!"
    assert 'FrameStack' in globals(), "FrameStack class not found - run the preprocessing cell first!"
    assert 'DQNAgent' in globals(), "DQNAgent class not found - run the DQNAgent cell first!"
    
    # Check imports
    import torch
    import gymnasium as gym
    import numpy as np
    import cv2
    from tqdm import tqdm
    
    print("✅ All classes and imports are ready!")
    print("✅ You can now run the training cell to start training!")
except AssertionError as e:
    print(f"❌ {e}")
except ImportError as e:
    print(f"❌ Missing import: {e}")
    print("Make sure you've run all the setup cells in order.")


✅ All classes and imports are ready!
✅ You can now run the training cell to start training!


In [16]:
import ale_py
import os
from tqdm import tqdm

# Create environment with preprocessing
env = gym.make("ALE/Pong-v5")
env = PreprocessAtari(env)
env = FrameStack(env, n_frames=4)

# Get environment info
state_shape = env.observation_space.shape
n_actions = env.action_space.n

print(f"State shape: {state_shape}")
print(f"Number of actions: {n_actions}")

# Create agent
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

agent = DQNAgent(
    state_shape=state_shape,
    n_actions=n_actions,
    device=device,
    lr=1e-4,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.01,
    epsilon_decay=10000,
    target_update_freq=1000,
    buffer_size=100000,
    batch_size=32
)

# Training parameters
total_steps = 1_000_000
learning_starts = 10_000  # Start training after this many steps
train_freq = 4  # Train every N steps
save_freq = 100_000  # Save model every N steps
eval_freq = 10_000  # Evaluate every N steps

# Statistics
episode_rewards = []
episode_lengths = []
current_episode_reward = 0
current_episode_length = 0

# Load existing model if available
model_path = "models/dqn_pong.pth"
if os.path.exists(model_path):
    agent.load(model_path)
    print(f"Loaded model from {model_path}")
else:
    os.makedirs("models", exist_ok=True)
    print("Starting fresh training")

# Training loop
state, info = env.reset()
pbar = tqdm(range(total_steps), desc="Training")

for step in pbar:
    # Select action
    action = agent.select_action(state, training=True)
    
    # Take step
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    
    # Store transition
    agent.store_transition(state, action, reward, next_state, done)
    
    # Update statistics
    current_episode_reward += reward
    current_episode_length += 1
    
    # Train
    if step >= learning_starts and step % train_freq == 0:
        loss = agent.train_step()
        if loss is not None:
            pbar.set_postfix({
                'epsilon': f'{agent.get_epsilon():.3f}',
                'loss': f'{loss:.4f}',
                'ep_reward': f'{current_episode_reward:.1f}' if done else 'N/A'
            })
    
    # Handle episode end
    if done:
        episode_rewards.append(current_episode_reward)
        episode_lengths.append(current_episode_length)
        current_episode_reward = 0
        current_episode_length = 0
        state, info = env.reset()
    else:
        state = next_state
    
    # Save model
    if step > 0 and step % save_freq == 0:
        agent.save(model_path)
        print(f"\nModel saved at step {step}")
    
    # Evaluate
    if step > 0 and step % eval_freq == 0 and len(episode_rewards) > 0:
        avg_reward = np.mean(episode_rewards[-10:]) if len(episode_rewards) >= 10 else np.mean(episode_rewards)
        avg_length = np.mean(episode_lengths[-10:]) if len(episode_lengths) >= 10 else np.mean(episode_lengths)
        print(f"\nStep {step}: Avg reward (last 10): {avg_reward:.2f}, Avg length: {avg_length:.1f}")

# Final save
agent.save(model_path)
env.close()
print(f"\nTraining complete! Model saved to {model_path}")
print(f"Total episodes: {len(episode_rewards)}")
if len(episode_rewards) > 0:
    print(f"Final average reward (last 100): {np.mean(episode_rewards[-100:]):.2f}")


State shape: (4, 84, 84)
Number of actions: 6
Using device: cuda
Loaded model from models/dqn_pong.pth


Training:   1%|          | 10045/1000000 [00:11<20:35, 801.30it/s, epsilon=0.010, loss=0.0304, ep_reward=N/A]


Step 10000: Avg reward (last 10): -21.00, Avg length: 782.0


Training:   2%|▏         | 20053/1000000 [00:34<40:23, 404.42it/s, epsilon=0.010, loss=0.0601, ep_reward=N/A]


Step 20000: Avg reward (last 10): -20.80, Avg length: 791.2


...


Step 980000: Avg reward (last 10): -21.00, Avg length: 770.0


Training:  99%|█████████▉| 990041/1000000 [36:43<00:23, 422.61it/s, epsilon=0.010, loss=0.0303, ep_reward=N/A]  


Step 990000: Avg reward (last 10): -21.00, Avg length: 776.0


Training: 100%|██████████| 1000000/1000000 [37:06<00:00, 449.13it/s, epsilon=0.010, loss=0.0303, ep_reward=N/A] 



Training complete! Model saved to models/dqn_pong.pth
Total episodes: 1276
Final average reward (last 100): -20.98


## Training Analysis & Answers to Your Questions

**Has it learned anything?**
Looking at your training output:
- Average reward: **-20.40 to -21.00** (basically random play - losing every point)
- For Pong, rewards range from -21 (lose all points) to +21 (win all points)
- **The agent hasn't learned much yet** - it's still playing randomly

**Should you increase to 1,000,000 steps?**
**YES!** DQN typically needs:
- **100,000 steps**: Just starting to learn (you're here)
- **500,000 steps**: Beginning to show improvement
- **1,000,000+ steps**: Actually learning to play well
- **10,000,000 steps**: Master-level play

**Why does it "learn so fast"?**
It's not actually learning fast - the **loss is decreasing** (which is good), but the **rewards aren't improving yet**. This is normal! The network is learning the Q-function, but it takes time for that to translate to better gameplay.

**Are we using a premade agent?**
**NO!** We built everything from scratch:
- ✅ Custom Model class (CNN architecture)
- ✅ Custom ReplayBuffer (experience replay)
- ✅ Custom DQNAgent (with target network, epsilon-greedy, etc.)
- ✅ Custom training loop

This is a **fully custom implementation** - not using stable-baselines3 or any pre-made agent!


In [15]:
# Evaluate the trained agent
import ale_py
import numpy as np

# Create evaluation environment
eval_env = gym.make("ALE/Pong-v5")
eval_env = PreprocessAtari(eval_env)
eval_env = FrameStack(eval_env, n_frames=4)

# Load the trained model
model_path = "models/dqn_pong.pth"
if os.path.exists(model_path):
    # Recreate agent with same parameters
    state_shape = eval_env.observation_space.shape
    n_actions = eval_env.action_space.n
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    eval_agent = DQNAgent(
        state_shape=state_shape,
        n_actions=n_actions,
        device=device,
        lr=1e-4,
        gamma=0.99,
        epsilon_start=1.0,
        epsilon_end=0.01,
        epsilon_decay=10000,
        target_update_freq=1000,
        buffer_size=100000,
        batch_size=32
    )
    eval_agent.load(model_path)
    print(f"✅ Loaded model from {model_path}")
    print(f"   Model was trained for {eval_agent.steps} steps")
else:
    print("❌ No model found to evaluate")
    eval_env.close()

# Run evaluation episodes
n_eval_episodes = 10
eval_rewards = []
eval_lengths = []

print(f"\nRunning {n_eval_episodes} evaluation episodes (no exploration, greedy policy)...")
for episode in range(n_eval_episodes):
    state, info = eval_env.reset()
    episode_reward = 0
    episode_length = 0
    
    while True:
        # Use greedy policy (no exploration)
        action = eval_agent.select_action(state, training=False)
        state, reward, terminated, truncated, info = eval_env.step(action)
        done = terminated or truncated
        
        episode_reward += reward
        episode_length += 1
        
        if done:
            break
    
    eval_rewards.append(episode_reward)
    eval_lengths.append(episode_length)
    print(f"Episode {episode+1}: Reward = {episode_reward:+.1f}, Length = {episode_length}")

eval_env.close()

# Print summary
print(f"\n{'='*50}")
print(f"Evaluation Summary ({n_eval_episodes} episodes):")
print(f"  Average Reward: {np.mean(eval_rewards):.2f}")
print(f"  Best Reward: {np.max(eval_rewards):.2f}")
print(f"  Worst Reward: {np.min(eval_rewards):.2f}")
print(f"  Average Length: {np.mean(eval_lengths):.1f} steps")
print(f"{'='*50}")

# Interpretation
avg_reward = np.mean(eval_rewards)
if avg_reward < -19:
    print("📉 Status: Still playing randomly (needs more training)")
    print("   → Increase training to 1,000,000+ steps")
elif avg_reward < -10:
    print("📈 Status: Starting to learn (showing some improvement)")
    print("   → Continue training to see more improvement")
elif avg_reward < 0:
    print("🎯 Status: Learning! (better than random)")
    print("   → Keep training to reach positive rewards")
elif avg_reward < 10:
    print("🏆 Status: Playing well! (winning some games)")
    print("   → Excellent progress!")
else:
    print("🌟 Status: Master level! (consistently winning)")


✅ Loaded model from models/dqn_pong.pth
   Model was trained for 100000 steps

Running 10 evaluation episodes (no exploration, greedy policy)...
Episode 1: Reward = -21.0, Length = 764
Episode 2: Reward = -21.0, Length = 764
Episode 3: Reward = -21.0, Length = 764
Episode 4: Reward = -21.0, Length = 764
Episode 5: Reward = -21.0, Length = 764
Episode 6: Reward = -21.0, Length = 764
Episode 7: Reward = -21.0, Length = 764
Episode 8: Reward = -21.0, Length = 764
Episode 9: Reward = -21.0, Length = 764
Episode 10: Reward = -21.0, Length = 764

Evaluation Summary (10 episodes):
  Average Reward: -21.00
  Best Reward: -21.00
  Worst Reward: -21.00
  Average Length: 764.0 steps
📉 Status: Still playing randomly (needs more training)
   → Increase training to 1,000,000+ steps


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    """
    DQN Model for Atari games.
    Takes stacked frames as input and outputs Q-values for each action.
    """
    def __init__(self, n_actions, n_frames=4):
        """
        Args:
            n_actions: Number of possible actions (e.g., 6 for Pong)
            n_frames: Number of stacked frames (default: 4)
        """
        super(Model, self).__init__()
        
        # Convolutional layers to process the image frames
        self.conv1 = nn.Conv2d(n_frames, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        
        # Calculate the size of the flattened feature map
        # Input shape: (n_frames, 84, 84) after preprocessing (or 210x160x3 raw)
        # After conv layers, we need to calculate the output size
        # For standard Atari preprocessing (84x84), the output is 7x7x64
        self.fc1 = nn.Linear(7 * 7 * 64, 512)
        self.fc2 = nn.Linear(512, n_actions)
        
    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x: Input tensor of shape (batch_size, n_frames, height, width)
        
        Returns:
            Q-values for each action, shape (batch_size, n_actions)
        """
        # Apply convolutional layers with ReLU activation
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Flatten the feature map
        x = x.view(x.size(0), -1)
        
        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x